**Disclaimer**

All codes in this calibration is taken either directly or heavily inspired by nicolainielsen32 on GitHub (linked as source in thesis). All computer vision codes are open source from the OpenCV library (also linked as source in thesis).

The codes are put in the repository to illustrate the process of developing a program, not to claim as the authors own work in its entirety.

Import packages

In [2]:
import numpy as np
import cv2
import os
import re

Collecting calibration

In [3]:
def undistortRectify(frameR, frameL):
    undistortedL= cv2.remap(frameL, stereoMapL_x, stereoMapL_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    undistortedR= cv2.remap(frameR, stereoMapR_x, stereoMapR_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    return undistortedR, undistortedL

Triangulation

In [4]:
def find_depth(right_point, left_point, frame_right, frame_left, baseline,f, alpha):

    # CONVERT FOCAL LENGTH f FROM [mm] TO [pixel]:
    height_right, width_right, depth_right = frame_right.shape
    height_left, width_left, depth_left = frame_left.shape

    if width_right == width_left:
        f_pixel = (width_right * 0.5) / np.tan(alpha * 0.5 * np.pi/180)

    else:
        print('Left and right camera frames do not have the same pixel width')
    x_right, _ = right_point[0]  # Extract x-coordinate from tuple
    x_left, _ = left_point[0]  # Extract x-coordinate from tuple

    # CALCULATE THE DISPARITY:
    disparity = x_left-x_right      #Displacement between left and right frames [pixels]

    # CALCULATE DEPTH z:
    zDepth = (baseline*f_pixel)/disparity             #Depth in [cm]

    return zDepth

def displaydepth(frameleft,frameright,centersl,centersr,B,f,alpha):
    if not centersr or not centersl:
        cv2.putText(frameright, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        cv2.putText(frameleft, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        return frameleft,frameright,"error no depth"
    else:
    # Function to calculate depth of object. Outputs vector of all depths in case of several balls.
    # All formulas used to find depth is in video presentaion
        depth = find_depth(centersr, centersl, frameright, frameleft, B, f, alpha)

        cv2.putText(frameright, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameleft, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameright, "Distance: " + str(round(depth,3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        cv2.putText(frameleft, "Distance: " + str(round(depth,3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0),2)
        # Multiply computer value with 205.8 to get real-life depth in [cm]. The factor was found manually.
        return frameleft,frameright,depth

depths = []   
def calculate_depths(frame_right,frame_left,baseline,f_pixel,alpha,folder_pathl,folder_pathr):

    def collect_coords(folder_path):
        files = os.listdir(folder_path)
        for file in files:
                file_path = sorted(os.path.join(folder_path, file))
                # Check if the file is a regular file (not a directory)
                if os.path.isfile(file_path):
                    # Extract coordinates from the file
                    coordinates = extract_coordinates(file_path)
                    print(coordinates)
        return coordinates

    left_coords = collect_coords(folder_pathl)
    right_coords = collect_coords(folder_pathr)
    for leftcoord, rightcoord in zip(left_coords, right_coords):
        # Calculate depth for the pair of points
        depth = find_depth(leftcoord, rightcoord, frame_right,frame_left,baseline,f_pixel,alpha)
        # Append the depth to the depths list
        depths.append(depth)

    return depths

Center detection

In [6]:
def masker(frame, lower, upper):
    mask = cv2.inRange(frame, lower, upper)
    mask = cv2.dilate(mask, None, iterations=2)
    return mask

def calculate_centers(contours):
    return [(x + w // 2, y + h // 2) for contour in contours for x, y, w, h in map(cv2.boundingRect, contours)]

def filter_centers(centers, colour,side,limit, index):
    try: 
        for i, center in enumerate(centers):
            res = f"./Week16/{colour}/res{side}{i}.txt"
            if index == 0:
                with open(res, "w") as file:
                    print(f"{center[0]},{center[1]}", file=file)
            else:
                with open(res, "r") as file:
                    lines = file.readlines()
                if lines:
                    prev_x, prev_y = map(float, lines[-1].strip().split(","))
                    if abs(center[0] - prev_x) <= limit and abs(center[1] - prev_y) <= limit:
                        with open(res, "a") as file:
                            print(f"{center[0]},{center[1]}", file=file)
    except FileNotFoundError:
        pass

def extract_coordinates(content):
    # Define a regular expression pattern to match coordinates
    pattern = r'(\d+),(\d+)'
    
    # Find all occurrences of coordinates in the content
    coordinates = re.findall(pattern, content)
    # Convert the coordinates from strings to integers
    coordinates = [(int(x), int(y)) for x, y in coordinates]

    return coordinates

def savecoords(coords,filename,index):
    mode = 'w' if index == 0 else 'a'
    with open(filename,mode) as file:
        form = " ".join([f"{x},{y}" for x, y in coords])
        file.write(f"{form}\n")
